In [1]:
from pyspark.sql import SQLContext
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.types import DateType
from datetime import date, timedelta
import datetime as dt
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import GBTRegressor
from pyspark.sql.functions import col, avg, sum
import numpy as np
import matplotlib.dates as mdates
from matplotlib import pyplot as plt
import pandas as pd
import pyspark

In [2]:
log4jLogger = sc._jvm.org.apache.log4j
LOGGER = log4jLogger.LogManager.getLogger(__name__)
LOGGER.error("pyspark script logger initialized")

## Read Feature File

In [3]:
sc.stop()
sc = pyspark.SparkContext(master="spark://172.16.27.208:7077",appName="spark")
sc

<SparkContext master=spark://172.16.27.208:7077 appName=spark>

In [4]:
print(sc.getConf().get("spark.default.parallelism"))

24


In [5]:
base_path = "/home/test5/Desktop/smart-meters-in-london/Feature_file/"
sqlcontext = SQLContext(sc)

In [6]:
df = []
for mth in range(1,13):
    month_df = sqlcontext.read.csv(base_path+"Cleaned_2013_Features_mth_{}.csv".format(mth),header = True,inferSchema=True,)
    if mth == 1:
        df = sqlcontext.createDataFrame(df,month_df.schema)
    df = df.union(month_df)

In [7]:
df = df.repartition(480,"LCLid")

In [8]:
df = df.drop("_c0")
df.printSchema()

root
 |-- LCLid: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- hour: integer (nullable = true)
 |-- energy(kWh/h): double (nullable = true)
 |-- month: integer (nullable = true)
 |-- weekDay: string (nullable = true)
 |-- 1_diff_energy_t_0: double (nullable = true)
 |-- 2_diff_energy_t_0: double (nullable = true)
 |-- 1_diff_energy_t_1: double (nullable = true)
 |-- 2_diff_energy_t_1: double (nullable = true)
 |-- 1_diff_energy_t_2: double (nullable = true)
 |-- 2_diff_energy_t_2: double (nullable = true)
 |-- diff_energy_week_t_1: double (nullable = true)
 |-- diff_energy_week_t_2: double (nullable = true)
 |-- diff_energy_week_t_3: double (nullable = true)
 |-- diff_energy_week_t_4: double (nullable = true)
 |-- mean_1: double (nullable = true)
 |-- mean_2: double (nullable = true)
 |-- min_1: double (nullable = true)
 |-- max_1: double (nullable = true)
 |-- min_2: double (nullable = true)
 |-- max_2: double (nullable = true)
 |-- count: integer (nullable = tr

## Disaggregated Approach using RF

## Perform :
    * 1 hot encoding
    * Vector Assembler

In [9]:
outputCols = ["weekDay_index","precipType_index","summary_index","stdorToU_index","Acorn_grouped_index"]
df_encoded = df
for col in outputCols: 
    encoder = OneHotEncoder(inputCol=col, outputCol="category_{}".format(col))
    df_encoded = encoder.transform(df_encoded).cache()
df_encoded.printSchema()

root
 |-- LCLid: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- hour: integer (nullable = true)
 |-- energy(kWh/h): double (nullable = true)
 |-- month: integer (nullable = true)
 |-- weekDay: string (nullable = true)
 |-- 1_diff_energy_t_0: double (nullable = true)
 |-- 2_diff_energy_t_0: double (nullable = true)
 |-- 1_diff_energy_t_1: double (nullable = true)
 |-- 2_diff_energy_t_1: double (nullable = true)
 |-- 1_diff_energy_t_2: double (nullable = true)
 |-- 2_diff_energy_t_2: double (nullable = true)
 |-- diff_energy_week_t_1: double (nullable = true)
 |-- diff_energy_week_t_2: double (nullable = true)
 |-- diff_energy_week_t_3: double (nullable = true)
 |-- diff_energy_week_t_4: double (nullable = true)
 |-- mean_1: double (nullable = true)
 |-- mean_2: double (nullable = true)
 |-- min_1: double (nullable = true)
 |-- max_1: double (nullable = true)
 |-- min_2: double (nullable = true)
 |-- max_2: double (nullable = true)
 |-- count: integer (nullable = tr

In [10]:
inputCols = ["weekDay","precipType","summary","stdorToU","Acorn_grouped","count"]
columns = df_encoded.columns
feature_col = columns[4:]
feature_col.append(columns[2])
feature_col = set(feature_col) - set(inputCols)
feature_col = feature_col - set(outputCols)
feature_col = list(feature_col)
feature_col.remove("date2")
len(feature_col)

34

In [11]:
# Dropping row with Na
df_encoded = df_encoded.na.drop()

In [12]:
vecAssembler = VectorAssembler(inputCols=feature_col, outputCol="features")
df_feature = vecAssembler.transform(df_encoded)
df_feature.take(1)

[Row(LCLid='MAC000439', date=datetime.datetime(2013, 1, 1, 0, 0), hour=0, energy(kWh/h)=0.684, month=1, weekDay='Tue', 1_diff_energy_t_0=0.804, 2_diff_energy_t_0=0.828, 1_diff_energy_t_1=0.928, 2_diff_energy_t_1=0.858, 1_diff_energy_t_2=0.871, 2_diff_energy_t_2=1.055, diff_energy_week_t_1=0.7, diff_energy_week_t_2=0.714, diff_energy_week_t_3=0.835, diff_energy_week_t_4=0.841, mean_1=0.6039166666666668, mean_2=0.5401666666666667, min_1=0.191, max_1=1.542, min_2=0.186, max_2=1.438, count=8760, visibility=13.28, windBearing=269, dewPoint=2.6, pressure=1008.19, apparentTemperature=3.66, windSpeed=5.46, precipType='rain', humidity=0.73, summary='Partly Cloudy', date2=datetime.datetime(2013, 1, 1, 0, 0), temperatureMax=7.49, temperatureMin=3.31, holiday=1, Weekday/end=0, stdorToU='ToU', Acorn_grouped='Comfortable', weekDay_index=4.0, precipType_index=0.0, summary_index=2.0, stdorToU_index=1.0, Acorn_grouped_index=2.0, category_weekDay_index=SparseVector(6, {4: 1.0}), category_precipType_inde

In [13]:
df_feature = df_feature.withColumnRenamed("energy(kWh/h)","label")
df_feature = df_feature.withColumn("date",df_feature["date"].cast(DateType()))
df_feature.printSchema()

root
 |-- LCLid: string (nullable = true)
 |-- date: date (nullable = true)
 |-- hour: integer (nullable = true)
 |-- label: double (nullable = true)
 |-- month: integer (nullable = true)
 |-- weekDay: string (nullable = true)
 |-- 1_diff_energy_t_0: double (nullable = true)
 |-- 2_diff_energy_t_0: double (nullable = true)
 |-- 1_diff_energy_t_1: double (nullable = true)
 |-- 2_diff_energy_t_1: double (nullable = true)
 |-- 1_diff_energy_t_2: double (nullable = true)
 |-- 2_diff_energy_t_2: double (nullable = true)
 |-- diff_energy_week_t_1: double (nullable = true)
 |-- diff_energy_week_t_2: double (nullable = true)
 |-- diff_energy_week_t_3: double (nullable = true)
 |-- diff_energy_week_t_4: double (nullable = true)
 |-- mean_1: double (nullable = true)
 |-- mean_2: double (nullable = true)
 |-- min_1: double (nullable = true)
 |-- max_1: double (nullable = true)
 |-- min_2: double (nullable = true)
 |-- max_2: double (nullable = true)
 |-- count: integer (nullable = true)
 |-- visi

##  actual and Predicted for given day

In [14]:
def get_aggregate(df):
    list = ["date","hour"]
    df = df.groupBy(list).agg(sum("label"),sum("prediction"))    
    return df

def select_predicted_actual(df,date,LCLid=None):
    list = []
    if LCLid != None:
        list = df.where((df["LCLid"] == LCLid) & (df["date"] == date)).select("label","prediction").collect()
    else:
        list = df.where((df["date"] == date)).select("label","prediction").collect()
    actual = [int(row['label']) for row in list]
    predicted = [int(row['prediction']) for row in list]
    return actual,predicted

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def root_mean_squared_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.sqrt(np.mean(np.abs((y_true - y_pred)**2)))

## Train-Test Split

In [16]:
train_df = df_feature.where(df_feature["date"] <= date(2013,10,31))
test_df = df_feature.where(df_feature["date"] > date(2013,10,31))# & (df_feature["date"] <= date(2013,1,2)))
#plits = train_df.randomSplit([0.01, 0.01,0.98], 24)
#splits[0].count()

print("Train_point = {}, Test_point = {}".format(train_df.count(),test_df.count()))

Train_point = 28641840, Test_point = 5753520


## RandomForest

In [ ]:
# rf = RandomForestRegressor(seed=123)
# paramGrid = ParamGridBuilder()\
#     .addGrid(rf.numTrees,[10,20,50]) \
#     .addGrid(rf.maxDepth,[10,15,20])\
#     .addGrid(rf.maxBins,[64,128])\
#     .addGrid(rf.featureSubsetStrategy,["auto","sqrt"])\
#     .build()
# tvs = TrainValidationSplit(estimator=rf,
#                            estimatorParamMaps=paramGrid,
#                            evaluator=RegressionEvaluator(),
#                            # 80% of the data will be used for training, 20% for validation.
#                            trainRatio=0.8)
# model = tvs.fit(train_df)

## Execute this 
rf = RandomForestRegressor(numTrees=20,maxDepth=10,maxBins=128,seed=4)
model = rf.fit(train_df)
best_model = model

In [ ]:
model.save(base_path+"rf_model_bin128")

In [ ]:
best_model = model.bestModel
print(" RF optimal num_trees = {}".format(best_model.getNumTrees))

In [ ]:
{param[0].name: param[1] for param in best_model.extractParamMap().items()}

In [ ]:
pred_val = best_model.transform(test_df)
pred_val.printSchema()
evaluator = RegressionEvaluator(labelCol='label', predictionCol='prediction', metricName="rmse")
accuracy = evaluator.evaluate(pred_val)

In [ ]:
print(accuracy)

## Gradient Boosted Tree

In [38]:
# gbt = GBTRegressor(seed=123)
# paramGrid = ParamGridBuilder()\
#     .addGrid(gbt.maxIter,[10,20,50]) \
#     .addGrid(gbt.maxDepth,[10,15,20])\
#     .addGrid(gbt.maxBins,[64,128])\
#     .build()
# tvs = TrainValidationSplit(estimator=gbt,
#                            estimatorParamMaps=paramGrid,
#                            evaluator=RegressionEvaluator(),
#                            # 80% of the data will be used for training, 20% for validation.
#                            trainRatio=0.8)
# gbt_model = tvs.fit(splits[0])

## Execute this 
gbt = GBTRegressor(maxBins=128,maxDepth=10)
gbt_model = gbt.fit(train_df)
best_model = gbt_model


In [39]:
gbt_model.save(base_path+"gbt_model")

In [37]:
{param[0].name: param[1] for param in gbt_model.bestModel.extractParamMap().items()}
best_model = gbt_model.bestModel

{'cacheNodeIds': False,
 'checkpointInterval': 10,
 'featuresCol': 'features',
 'impurity': 'variance',
 'labelCol': 'label',
 'lossType': 'squared',
 'maxBins': 128,
 'maxDepth': 10,
 'maxIter': 10,
 'maxMemoryInMB': 256,
 'minInfoGain': 0.0,
 'minInstancesPerNode': 1,
 'predictionCol': 'prediction',
 'seed': 123,
 'stepSize': 0.1,
 'subsamplingRate': 1.0}

In [41]:
pred_val = best_model.transform(test_df)
pred_val.printSchema()
evaluator = RegressionEvaluator(labelCol='label', predictionCol='prediction', metricName="rmse")
accuracy = evaluator.evaluate(pred_val)

root
 |-- LCLid: string (nullable = true)
 |-- date: date (nullable = true)
 |-- hour: integer (nullable = true)
 |-- label: double (nullable = true)
 |-- month: integer (nullable = true)
 |-- weekDay: string (nullable = true)
 |-- 1_diff_energy_t_0: double (nullable = true)
 |-- 2_diff_energy_t_0: double (nullable = true)
 |-- 1_diff_energy_t_1: double (nullable = true)
 |-- 2_diff_energy_t_1: double (nullable = true)
 |-- 1_diff_energy_t_2: double (nullable = true)
 |-- 2_diff_energy_t_2: double (nullable = true)
 |-- diff_energy_week_t_1: double (nullable = true)
 |-- diff_energy_week_t_2: double (nullable = true)
 |-- diff_energy_week_t_3: double (nullable = true)
 |-- diff_energy_week_t_4: double (nullable = true)
 |-- mean_1: double (nullable = true)
 |-- mean_2: double (nullable = true)
 |-- min_1: double (nullable = true)
 |-- max_1: double (nullable = true)
 |-- min_2: double (nullable = true)
 |-- max_2: double (nullable = true)
 |-- count: integer (nullable = true)
 |-- visi

In [42]:
print(accuracy)

0.38920046777462886


In [ ]:
## COmmon Method now on

In [ ]:
pred_val.select("LCLid","label","prediction").show()

In [ ]:
aggregate_df = get_aggregate(pred_val)
aggregate_df = aggregate_df.withColumnRenamed("sum(label)","label")
aggregate_df = aggregate_df.withColumnRenamed("sum(prediction)","prediction")

In [ ]:
aggregate_df.select("label","prediction").show()

## Own Metric

In [ ]:
train_start_date = date(2013,11,1)
train_end_date = date(2013,12,31)
y_date = []
Mape_date = []
rmse_date = []
while train_start_date <= train_end_date:
    print(train_start_date)
    y_actual,y_pred = select_predicted_actual(aggregate_df,train_start_date)
    if len(y_actual) == 0:
        train_start_date = train_start_date + timedelta(1)
        continue
    Mape_date.append(mean_absolute_percentage_error(y_actual,y_pred))
    rmse_date.append(root_mean_squared_error(y_actual,y_pred))
    y_date.append(train_start_date)
    train_start_date = train_start_date + timedelta(1)

In [ ]:
#del y_date[2]
# fig, (ax1,ax2) = plt.subplots(1,2, figsize =(8,6))
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%m/%d/%Y'))
# ax1.xaxis.set_major_locator(mdates.DayLocator())
# ax2.xaxis.set_major_formatter(mdates.DateFormatter('%m/%d/%Y'))
# ax2.xaxis.set_major_locator(mdates.DayLocator())
# ax1.plot(y_date,Mape_date)
# ax2.plot(y_date,rmse_date)
# fig.autofmt_xdate()
# ax1.set_xlabel('k')
# ax1.set_ylabel('cost')
date_time = pd.to_datetime(y_date)
DF = pd.DataFrame()
DF['Mape_date'] = Mape_date
DF = DF.set_index(date_time)

DF1 = pd.DataFrame()
DF1['rmse_date'] = rmse_date
DF1 = DF1.set_index(date_time)
fig, (ax1,ax2) = plt.subplots(1,2, figsize =(8,6))
fig.subplots_adjust(bottom=0.3)
for ax in fig.axes:
    plt.sca(ax)
    plt.xticks(rotation=90)
ax1.plot(DF)
ax2.plot(DF1)

In [ ]:
from statistics import mean
print("Mean RMSE = {}, Mean Mape = {}".format(mean(rmse_date),mean(Mape_date)))

In [ ]:
fig.savefig(base_path+"rf_bin_128")